In [ ]:
import numpy as np

In [ ]:
import nltk

In [ ]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [ ]:
from nltk.corpus import gutenberg

In [ ]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [ ]:
text = ''
for txt in gutenberg.fileids():
  if 'shakespeare' in txt:
    text = text + gutenberg.raw(txt).lower()

print('corpus length: ', len(text))

corpus length:  375542


In [ ]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars:  50


In [ ]:
print(text[:500])

[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a carpenter

   mur. where is thy leather apron, and thy rule?
what dost thou with thy best apparrell on


In [ ]:
maxlen = 400
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])

print('nb sequences: ', len(sentences))

nb sequences:  125048


In [ ]:
print('Vectorization ......... ')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    X[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

Vectorization ......... 


In [ ]:
sentences[0]

'[the tragedie of julius caesar by william shakespeare 1599]\n\n\nactus primus. scoena prima.\n\nenter flauius, murellus, and certaine commoners ouer the stage.\n\n  flauius. hence: home you idle creatures, get you home:\nis this a holiday? what, know you not\n(being mechanicall) you ought not walke\nvpon a labouring day, without the signe\nof your profession? speake, what trade art thou?\n  car. why sir, a ca'

In [ ]:
sentences[0][1]

't'

In [ ]:
X[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False,  True, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
text[0:4]

'[the'

In [ ]:
for i, sentence in enumerate(sentences):
  if i == 0:
    print(sentence)

[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a ca


In [ ]:
next_chars[1]

'n'

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM 
from keras.optimizers import RMSprop  

In [ ]:
print('Build Model ...... ')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

print(model.summary())

Build Model ...... 
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               91648     
_________________________________________________________________
dense (Dense)                (None, 50)                6450      
_________________________________________________________________
activation (Activation)      (None, 50)                0         
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 6
batch_size = 128

model_structure = model.to_json()
with open("shakes_lstm_model.json", "w") as json_file:
  json_file.write(model_structure)

for i in range(5):
  model.fit(X, y, batch_size=batch_size, epochs=epochs)

Epoch 1/6
977/977 [==============================] - 917s 939ms/step - loss: 2.0534
Epoch 2/6
977/977 [==============================] - 915s 936ms/step - loss: 1.7060
Epoch 3/6
977/977 [==============================] - 915s 937ms/step - loss: 1.5940
Epoch 4/6
977/977 [==============================] - 915s 937ms/step - loss: 1.5258
Epoch 5/6
977/977 [==============================] - 915s 937ms/step - loss: 1.4815
Epoch 6/6
977/977 [==============================] - 912s 934ms/step - loss: 1.4475
Epoch 1/6
 86/977 [=>............................] - ETA: 13:41 - loss: 1.3652

KeyboardInterrupt: ignored

In [ ]:
model.save_weights("shakes_lstm_weights_{}.h5".format(i+1))
print('Model Saved .... ')

Model Saved .... 


In [ ]:
len(chars)

50

In [ ]:
import random

In [ ]:
def sample(preds, temperature=1.0):
  # helper function to sample an index from a probability array
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
import sys

start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0]:
  print()
  print('--- diversity:', diversity)

  generated = ''
  sentence = text[start_index: start_index + maxlen]
  generated = generated + sentence 
  print(' -------- Generating with seed: "' + sentence + '"')
  sys.stdout.write(generated)

  for i in range(400):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x[0, t, char_indices[char]] == 1.

    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]

    generated = generated + next_char 
    sentence = sentence[1:] + next_char 

    sys.stdout.write(next_char)
    sys.stdout.flush()

print()


--- diversity: 0.2
 -------- Generating with seed: "
  polon. if you call me iephta my lord, i haue a daughter
that i loue passing well

   ham. nay that followes not

   polon. what followes then, my lord?
  ha. why, as by lot, god wot: and then you know, it
came to passe, as most like it was: the first rowe of the
pons chanson will shew you more. for looke where my
abridgements come.
enter foure or fiue players.

y'are welcome masters, welcome al"

  polon. if you call me iephta my lord, i haue a daughter
that i loue passing well

   ham. nay that followes not

   polon. what followes then, my lord?
  ha. why, as by lot, god wot: and then you know, it
came to passe, as most like it was: the first rowe of the
pons chanson will shew you more. for looke where my
abridgements come.
enter foure or fiue players.

y'are welcome masters, welcome al.................................................................................................................................................

In [ ]:
type(start_index)

int

In [ ]:
type(maxlen)

int

In [ ]:
type(start_index + maxlen)

int